## Pré-Processamento 

**Pensamentos Iniciais:** temos um arquivo .txt de contos, crônicas e o romance *A Hora da Estrela*, todos de autoria de Clarice Lispector. Como o projeto de entrada requer a utilização do Word2Vec e do BERT, precisaremos pegar o arquivo "cru" do texto e fazer uma série de modificações:

   1. Eliminar letras maiúsculas
    
   2. Transformar o texto em um conjunto de sentenças
    
   3. Remover *stopwords* e pontuações 
    
   4. Tokenizar e lemmatizar o texto
    
Assim, ao final do pré-processamento, teremos uma lista "result" composta de um conjunto de listas (as sentenças do texto), que, por sua vez, são compostas de um conjunto de palavras (tokens).

In [1]:
#Importando bibliotecas
import pandas as pd
import spacy
import re, string
import gensim

In [2]:
#Abrindo o arquivo
with open("contos.txt", encoding="UTF-8") as f:
    contos = f.read()
contos = contos.lower()
with open("estrela.txt", encoding="UTF-8") as f:
    estrela = f.read()
estrela = estrela.lower()
with open("cronicas.txt", encoding="UTF-8") as f:
    cronicas = f.read()
cronicas = cronicas.lower()
print("Tamanho do arquivo de contos: ", len(contos))
print("Tamanho do arquivo da obra A Hora da Estrela: ", len(estrela))
print("Tamanho do arquivo de crônicas: ", len(cronicas))
print(contos[0:100])
print(estrela[0:100])
print(cronicas[0:104])

Tamanho do arquivo de contos:  910751
Tamanho do arquivo da obra A Hora da Estrela:  129364
Tamanho do arquivo de crônicas:  1079263
primeiras histórias
o triunfo
o relógio bate 9 horas. uma pancada alta, sonora, seguida de uma badal
      tudo no mundo começou com um sim. uma molécula disse sim
a outra molécula e nasceu a vida. mas
nem uma vírgula
aquela    sexta-feira 18 de agosto de 1967 foi especialmente tensa na
redação do caderno


In [3]:
text = estrela + contos + cronicas

In [4]:
nlp = spacy.load("pt_core_news_sm")
nlp.max_length = 2200000 

In [5]:
#Passando o arquivo para o spacy
doc = nlp(text)

In [6]:
#Transformando o arquivo em uma lista de sentenças 
sentences = list(doc.sents)

In [7]:
#Finalizando o pré-processamento
result = []
token_classe = []
for sent in sentences:
    token_result = []
    for token in sent:
        if not token.is_punct and not token.is_stop:
            if token.text != "\n" and token.text != "\n\n\x0c" and token.text !="\n\n":
                token_classe.append(token.pos_)
                token_result.append(token.lemma_)
    result.append(token_result)
print(result)

[['      ', 'mundo', 'começar'], ['molécula', 'dizer', 'molécula', 'nascer', 'vidar'], ['pré-história', 'haver', 'pré-história', 'pré-história', 'haver', 'haver'], ['haver'], ['universo', 'jamais', 'começar'], ['\n      ', 'ninguém', 'enganir', 'conseguir', 'simplicidade', 'trabalhar'], ['\n      ', 'ter', 'perguntar', 'haver', 'respostar', 'continuar', 'escrever'], ['começar', 'início', 'coisa', 'acontecer', 'acontecer'], ['pré-pré-história', 'haver', 'monstro', 'apocalíptico'], ['história', 'existir', 'passar', 'existir'], ['pensar', 'atar'], ['sentir', 'fato'], ['junto', 'escrever', 'escrever'], ['deus', 'mundo'], ['verdade', 'contato', 'interior', 'inexplicável'], ['vidar', 'verdadeiro', 'irreconhecível', 'extremamente', 'interior', 'palavra', 'significar'], ['coração', 'esvaziar', 'desejar', 'reduz-se', 'pulsar'], ['dor', 'dente', 'perpassar', 'história', 'dar', 'fisgar', 'fundo', 'pleno', 'bocar'], ['cantar', 'alto', 'agudo', 'melodiar', 'sincopar', 'estridente', 'próprio', 'dor'

**Observações Pré-Processamento**: de um modo geral, o pré-processamento foi bem-feito. No entanto, alguns probleminhas podem ser encontrados, sobretudo na parte de lemmatização: tanto "casa" quanto "casar" são lemmatizados para "casar", o que não é algo desejável já que as duas palavras têm significados distintos. 

## Exploração do Corpus

**Ideia Central:** conforme está escrito no enunciado do projeto de ingresso, a ideia central por trás da análise inicial é explorar um pouco o corpus, checando termos mais frequentes, classes gramaticais mais frequentes, tamanho da análise, etc

In [8]:
#Número de Termos do texto processado
from collections import defaultdict  
term_freq = defaultdict(int)
for sent in result:
    for i in sent:
        term_freq[i] += 1
print(len(term_freq))
print(len(result))

15664
33448


**Tamanho da Análise**: é possível observar que há 15664 termos distintos nos textos de Clarice, mostrando a diversidade de linguagem da autora.

In [9]:
#Termos mais frequentes
sorted(term_freq, key=term_freq.get, reverse=True)[:10]

['\n   ',
 'ter',
 'ser',
 'dizer',
 'haver',
 'vidar',
 'homem',
 'ficar',
 'sentir',
 'dia']

**Termos mais Frequentes**: a escrita de Clarice é caracterizada por ser intimista, pessoal, focando na intimidade do personagem. Logo, é interessante ressaltar a presença de palavras não-verbais na lista de termos mais frequentes, como "vida", "homem" e "dia", que retratam justamente esse foco narrativo no cotidiano, no íntimo.

In [10]:
#Classes gramaticais mais frequentes
classe_freq = defaultdict(int)
for classe in token_classe:
    classe_freq[classe] += 1
print(classe_freq)

defaultdict(<class 'int'>, {'SPACE': 4387, 'NOUN': 69991, 'VERB': 52343, 'ADJ': 20255, 'ADV': 4909, 'PRON': 2832, 'AUX': 2817, 'ADP': 551, 'DET': 1499, 'NUM': 583, 'PROPN': 3715, 'SCONJ': 101, 'X': 45, 'INTJ': 175, 'CCONJ': 18, 'PART': 1, 'PUNCT': 11, 'SYM': 1})


**Classes Gramaticais mais frequentes**: conforme esperado, as classes gramaticais mais frequentes são verbos e substantivos. Ainda assim, é destacável a presença significativa de adjetivos nos textos de Clarice, que ajudam a compor a atmosfera intimista de seu estilo literário.

## Word2Vec  

**Ideia Central:** a transformação de palavras em vetores será feita utilizando duas bases de dados: primeiro, essa transformação será feita utilizando os dados pré-processados dos textos de Lispector; depois, será realizada com um modelo do NILC, já disponível na biblioteca gensim. Para ambos os casos, o passo-a-passo é o seguinte:
    
   1. Montar o modelo, definindo os melhores parâmetros a serem adotados (exemplo: mínima frequência de cada palavra, tamanho da amostra, etc)
   2. Criar o vocabulário a ser usado no modelo (basicamente as palavras presentes na lista *result*)
   3. Treinar o modelo
   4. Obter relações de similaridades e analogias entre as palavras, usando similaridade de cossenos 

In [82]:
import multiprocessing

from gensim.models import Word2Vec

In [83]:
cores = multiprocessing.cpu_count() 

In [121]:
w2v_model = Word2Vec(min_count=50,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.001, 
                     negative=2,
                     workers=cores-1)

In [122]:
w2v_model.build_vocab(result, progress_per=50000)

In [123]:
w2v_model.train(result, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

(395988, 3284680)

### Parte 1: Metodologia 

Após o modelo ter sido criado e treinado, podemos obter relações de similaridade e analogia entre as palavras. Para tanto, foi usada a similaridade de cossenos. Uma outra possibilidade seria calcular a similaridade com base na distância euclidiana dos vetores, mas, como a melhor forma de se descobrir quão semelhantes são dois vetores é a partir da diferença de direção deles, a similaridade de cossenos parece ser a forma mais adequada de avaliar o modelo.

Nesse sentido, serão feitos três testes: primeiro, testaremos a similaridade de duas palavras; depois, escolheremos uma palavra e deixaremos a máquina avaliar quais são os termos mais similares à palavra de escolha; por fim, escolheremos dois termos semânticamente próximos e um terceiro termo mais distante dos dois primeiros e avaliaremos se a máquina é capaz de distinguir o "patinho feio".

### Parte 2: Similaridade de duas palavras

In [124]:
w2v_model.wv.similarity("macabéa", 'glória')

0.89598656

In [131]:
w2v_model.wv.similarity("mulher", "pessoa")

0.97625625

**Conclusão parte 2**: o modelo indica que Macabéa e Glória tem similaridade de cerca de 90%, o que parece fazer sentido, dado que as personagens dialogam constantemente na obra de Lispector. Todavia, a similaridade entre mulher e pessoa, de 97%, parece ser um pouco exagerada, dado que a mulher não é o único tipo de pessoa que existe. Uma possível explicação para esse valor exagerado é o tamanho do corpus, que é muito menor e menos diverso do que aqueles comumente utilizados para esse tipo de análise.

### Parte 3: Top10 palavras mais similares

In [126]:
w2v_model.wv.most_similar(positive=["moço"])

[('ser', 0.9722307324409485),
 ('mulher', 0.9704183340072632),
 ('ter', 0.9681292176246643),
 ('haver', 0.9668287038803101),
 ('quase', 0.9654912352561951),
 ('vidar', 0.9652710556983948),
 ('escrever', 0.9650448560714722),
 ('chegar', 0.9641159772872925),
 ('dizer', 0.9620192646980286),
 ('passar', 0.9613848924636841)]

In [127]:
w2v_model.wv.most_similar(positive=["história"])

[('contar', 0.9436878561973572),
 ('haver', 0.9378396272659302),
 ('próprio', 0.9372369050979614),
 ('homem', 0.9369253516197205),
 ('passar', 0.9359861016273499),
 ('olho', 0.9354768991470337),
 ('dia', 0.9352109432220459),
 ('algum', 0.9350494146347046),
 ('ficar', 0.9347135424613953),
 ('quase', 0.9338269233703613)]

In [128]:
w2v_model.wv.most_similar(positive=["macabéa"])

[('quase', 0.9372661709785461),
 ('pessoa', 0.9345123767852783),
 ('ter', 0.9340687394142151),
 ('estar', 0.9321044683456421),
 ('coração', 0.9320266246795654),
 ('andar', 0.9309702515602112),
 ('janela', 0.9294223189353943),
 ('dia', 0.929111123085022),
 ('moço', 0.9280498623847961),
 ('mundo', 0.9275633096694946)]

**Conclusão Parte 3**: os modelos não são muito precisos. Moço, por exemplo, apresenta como termos mais próximos verbos comuns como "ser", "ter" e "haver". Mesmo assim, cabe o destaque de que o termo mais semelhante à "história" é "contar", o que faz muito sentido, e que termos sentimentais como "coração" e do cotidiano como "janela", "dia" e "mundo" estão associados a "Macabéa".

### Parte 4: Patinho Feio

In [129]:
w2v_model.wv.doesnt_match(['cabeça', 'olho', 'cama'])

'cama'

In [130]:
w2v_model.wv.doesnt_match(['dinheiro', 'poder', 'pé'])

'dinheiro'

**Conclusão Parte 4**: novamente, o modelo apresenta erros e acertos. Corretamente, distinguiu que "cama", por não ser uma parte do corpo humano, era o "patinho feio" em relação a "cabeça" e "olho". Porém, na lista composta por dinheiro, poder e pé, a máquina interpretou que o termo a ser excluído seria "dinheiro" ao invés de "pé", mostrando a imprecisão do modelo.

### Conclusão Geral - Clarice 

Ainda que o pré-processamento tenha sido feito adequadamente, os resultados obtidos com o Word2Vec não foram os mais satisfatórios, muito em função do tamanho reduzido da amostra para a vetorização e a análise adequada das palavras.